In [1]:
!pip install --no-warn-conflicts -q deepctr==0.7.5

     |████████████████████████████████| 92kB 3.0MB/s 


In [0]:
from google.colab import (files,drive)
import os
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model, load_model
from deepctr.models import DIN, DeepFM
from deepctr.inputs import SparseFeat,VarLenSparseFeat,DenseFeat,get_feature_names
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.layers import Activation
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks
from tensorflow.keras import utils
import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
import pandas_profiling 
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
warnings.simplefilter('ignore')

In [3]:
drive.mount('/content/gdrive', force_remount=True)
path = "/content/gdrive/"
os.chdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#The original data consists of four tables with raw_sample, ad_features, user_profiles and user_behaviors. We joined the all tables on big query and selected part of data for samples
data_0 = pd.read_csv('My Drive/BAX423+453 Five Guys/BAX423 BIG DATA/Final Project/Data/merge_data/merge_data_10k_0.csv')
data_1 = pd.read_csv('My Drive/BAX423+453 Five Guys/BAX423 BIG DATA/Final Project/Data/merge_data/merge_data_10k_1.csv')
data_2 = pd.read_csv('My Drive/BAX423+453 Five Guys/BAX423 BIG DATA/Final Project/Data/merge_data/merge_data_10k_2.csv')
data_3 = pd.read_csv('My Drive/BAX423+453 Five Guys/BAX423 BIG DATA/Final Project/Data/merge_data/merge_data_10k_3.csv')
data_4 = pd.read_csv('My Drive/BAX423+453 Five Guys/BAX423 BIG DATA/Final Project/Data/merge_data/merge_data_10k_4.csv')

In [0]:
data = pd.concat([data_0,data_1,data_2,data_3,data_4], ignore_index=True, sort=False)

In [6]:
# drop overlaps with 'user' and useless variables
final_df = data.drop(columns=['userid','cms_segid', 'cms_group_id', 'customer', 'ad_brand', 'nonclk','pid','ad_time_stamp'])
final_df.head()

,user,cate,btag,num,adgroup_id,final_gender_code,age_level,pvalue_level,shopping_level,occupation,new_user_class_level_,clk,ad_cate_id,campaign_id,price
0,750820,6300,pv,3,383310,1,6,3.0,3,0,3.0,0,6261,289341,6599.0
1,301656,4609,pv,7,575898,1,6,2.0,3,0,3.0,0,6250,232372,78.0
2,750820,4521,pv,39,745756,1,6,3.0,3,0,3.0,0,6261,154448,665.0
3,750820,5957,pv,3,609938,1,6,3.0,3,0,3.0,0,6246,355648,128.0
4,750820,4281,pv,25,677651,1,6,3.0,3,0,3.0,0,6261,304098,1018.0


### Deep Factorization Machines
 Factorization machines (FMs) are a supervised learning approach that embed features into a latent space and model
the interactions between features via inner product of their
embedding vectors. Models based on degree-2 polynomial
mapping and factorization machines are widely used for
CTR prediction. Deep FM model combines the power of factorization machines for recommendation and deep learning for feature learning in a new neural network architecture.

In [0]:
#Now let us choose the sparse,dense required for FM model as shown below
sparse_features = ['user','cate', 'btag','adgroup_id', 'final_gender_code', 'age_level','pvalue_level', 'shopping_level', 
                   'occupation', 'new_user_class_level_','campaign_id','ad_cate_id']
dense_features = ['num','price']

In [0]:
final_df[sparse_features] = final_df[sparse_features].fillna('-1', )
final_df[dense_features] = final_df[dense_features].fillna(0, )
target = ['clk']

In [0]:
#Convert columns to str for label enocding
final_df['pvalue_level'] = final_df['pvalue_level'].astype(str)
final_df['new_user_class_level_'] = final_df['new_user_class_level_'].astype(str)

In [0]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
  le = LabelEncoder()
  final_df[feat] = le.fit_transform(final_df[feat])
mms = MinMaxScaler(feature_range=(0, 1))
final_df[dense_features] = mms.fit_transform(final_df[dense_features])

In [0]:
# 2.count #unique features for each sparse field,and record dense feature field name
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=final_df[feat].nunique(),embedding_dim=4)
                          for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                          for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [0]:
# 3.generate input data for model
train, test = train_test_split(final_df, test_size=0.2)
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

In [13]:
# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )

history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2, )
pred_ans = model.predict(test_model_input, batch_size=256)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

Epoch 1/10
31561/31561 - 335s - loss: 0.0753 - binary_crossentropy: 0.0681 - val_loss: 0.0532 - val_binary_crossentropy: 0.0460
Epoch 2/10
31561/31561 - 329s - loss: 0.0508 - binary_crossentropy: 0.0437 - val_loss: 0.0497 - val_binary_crossentropy: 0.0426
Epoch 3/10
31561/31561 - 325s - loss: 0.0489 - binary_crossentropy: 0.0418 - val_loss: 0.0480 - val_binary_crossentropy: 0.0409
Epoch 4/10
31561/31561 - 327s - loss: 0.0486 - binary_crossentropy: 0.0414 - val_loss: 0.0489 - val_binary_crossentropy: 0.0418
Epoch 5/10
31561/31561 - 329s - loss: 0.0482 - binary_crossentropy: 0.0411 - val_loss: 0.0486 - val_binary_crossentropy: 0.0415
Epoch 6/10
31561/31561 - 331s - loss: 0.0479 - binary_crossentropy: 0.0408 - val_loss: 0.0477 - val_binary_crossentropy: 0.0408
Epoch 7/10
31561/31561 - 331s - loss: 0.0477 - binary_crossentropy: 0.0407 - val_loss: 0.0483 - val_binary_crossentropy: 0.0413
Epoch 8/10
31561/31561 - 330s - loss: 0.0476 - binary_crossentropy: 0.0406 - val_loss: 0.0474 - val_bina